# LSTM-arithmetic

## Dataset
- [Arithmetic dataset](https://drive.google.com/file/d/1cMuL3hF9jefka9RyF4gEBIGGeFGZYHE-/view?usp=sharing)

In [1]:
# ! pip install seaborn
# ! pip install opencc
# ! pip install -U scikit-learn

import numpy as np
import pandas as pd
import torch
import torch.nn
import torch.nn.utils.rnn
import torch.utils.data
import matplotlib.pyplot as plt
import seaborn as sns
import opencc
import os
from sklearn.model_selection import train_test_split

data_path = './data'

In [2]:
df_train = pd.read_csv(os.path.join(data_path, 'arithmetic_train.csv'))
df_eval = pd.read_csv(os.path.join(data_path, 'arithmetic_eval.csv'))
df_train.head()

,Unnamed: 0,src,tgt
0,2285313,14*(43+20)=,882
1,317061,(6+1)*5=,35
2,718770,13+32+29=,74
3,170195,31*(3-11)=,-248
4,2581417,24*49+1=,1177


In [3]:
# transform the input data to string
df_train['tgt'] = df_train['tgt'].apply(lambda x: str(x))
df_train['src'] = df_train['src'].add(df_train['tgt'])
df_train['len'] = df_train['src'].apply(lambda x: len(x))

df_eval['tgt'] = df_eval['tgt'].apply(lambda x: str(x))
df_eval['src'] = df_eval['src'].add(df_eval['tgt'])
df_eval['len'] = df_eval['src'].apply(lambda x: len(x))

# Build Dictionary
 - The model cannot perform calculations directly with plain text.
 - Convert all text (numbers/symbols) into numerical representations.
 - Special tokens
    - '&lt;pad&gt;'
        - Each sentence within a batch may have different lengths.
        - The length is padded with '&lt;pad&gt;' to match the longest sentence in the batch.
    - '&lt;eos&gt;'
        - Specifies the end of the generated sequence.
        - Without '&lt;eos&gt;', the model will not know when to stop generating.

In [4]:
char_to_id = {}
id_to_char = {}

# write your code here
# Build a dictionary and give every token in the train dataset an id
# The dictionary should contain <eos> and <pad>
# char_to_id is to conver charactors to ids, while id_to_char is the opposite
special_tokens = ['<pad>', '<eos>']
all_chars = set(''.join(df_train['src']) + ''.join(df_train['src']))

# Separate digits and symbols
digits = sorted([char for char in all_chars if char.isdigit()])
symbols = sorted([char for char in all_chars if not char.isdigit()])

# Assign ids to special tokens first
char_to_id = {'<pad>': 0, '<eos>': 1}

# Assign ids to digits and symbols
char_to_id.update({char: idx + 2 for idx, char in enumerate(digits + symbols)})
id_to_char = {idx: char for char, idx in char_to_id.items()}
#print(id_to_char)
vocab_size = len(char_to_id)
print('Vocab size: {}'.format(vocab_size))

Vocab size: 18


# Data Preprocessing
 - The data is processed into the format required for the model's input and output.
 - Example: 1+2-3=0
     - Model input: 1 + 2 - 3 = 0
     - Model output: / / / / / 0 &lt;eos&gt;  (the '/' can be replaced with &lt;pad&gt;)
     - The key for the model's output is that the model does not need to predict the next character of the previous part. What matters is that once the model sees '=', it should start generating the answer, which is '0'. After generating the answer, it should also generate&lt;eos&gt;


In [5]:
df_train['char_id_list'] = df_train['src'].apply(lambda x: [char_to_id[char] for char in x] + [char_to_id['<eos>']])
df_train.head()


,Unnamed: 0,src,tgt,len,char_id_list
0,2285313,14*(43+20)=882,882,14,"[3, 6, 14, 12, 6, 5, 15, 4, 2, 13, 17, 10, 10,..."
1,317061,(6+1)*5=35,35,10,"[12, 8, 15, 3, 13, 14, 7, 17, 5, 7, 1]"
2,718770,13+32+29=74,74,11,"[3, 5, 15, 5, 4, 15, 4, 11, 17, 9, 6, 1]"
3,170195,31*(3-11)=-248,-248,14,"[5, 3, 14, 12, 5, 16, 3, 3, 13, 17, 16, 4, 6, ..."
4,2581417,24*49+1=1177,1177,12,"[4, 6, 14, 6, 11, 15, 3, 17, 3, 3, 9, 9, 1]"


In [7]:
import numpy as np

# 提前計算 len 的最大值
max_len = df_train['len'].max()

# 建立 src 和 tgt 的 char_id_list 和 label_id_list
src_char_id_list = []
label_id_list = []

for row in df_train.itertuples(index=False):
    src_char_ids = [char_to_id[char] for char in row.src] + [char_to_id['<eos>']]
    tgt_char_ids = [char_to_id['<pad>']] * (max_len - len(row.tgt)) + [char_to_id[char] for char in row.tgt] + [char_to_id['<eos>']]
    
    src_char_id_list.append(src_char_ids)
    label_id_list.append(tgt_char_ids)

# 重新構建 DataFrame
df_train['char_id_list'] = src_char_id_list
df_train['label_id_list'] = label_id_list

df_train.head()


,Unnamed: 0,src,tgt,len,char_id_list,label_id_list
0,2285313,14*(43+20)=882,882,14,"[3, 6, 14, 12, 6, 5, 15, 4, 2, 13, 17, 10, 10,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 10, 10..."
1,317061,(6+1)*5=35,35,10,"[12, 8, 15, 3, 13, 14, 7, 17, 5, 7, 1]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 5, ..."
2,718770,13+32+29=74,74,11,"[3, 5, 15, 5, 4, 15, 4, 11, 17, 9, 6, 1]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 9, ..."
3,170195,31*(3-11)=-248,-248,14,"[5, 3, 14, 12, 5, 16, 3, 3, 13, 17, 16, 4, 6, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 16, 4, 6,..."
4,2581417,24*49+1=1177,1177,12,"[4, 6, 14, 6, 11, 15, 3, 17, 3, 3, 9, 9, 1]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 3, 9, ..."


In [8]:
# 提前計算 df_eval['len'] 的最大值
max_len_eval = df_eval['len'].max()

# 建立 src 和 tgt 的 char_id_list 和 label_id_list
src_char_id_list_eval = []
label_id_list_eval = []

for row in df_eval.itertuples(index=False):
    src_char_ids = [char_to_id[char] for char in row.src] + [char_to_id['<eos>']]
    tgt_char_ids = [char_to_id['<pad>']] * (max_len_eval - len(row.tgt)) + [char_to_id[char] for char in row.tgt] + [char_to_id['<eos>']]
    
    src_char_id_list_eval.append(src_char_ids)
    label_id_list_eval.append(tgt_char_ids)

# 重新構建 DataFrame
df_eval['char_id_list'] = src_char_id_list_eval
df_eval['label_id_list'] = label_id_list_eval

df_eval.head()


,Unnamed: 0,src,tgt,len,char_id_list,label_id_list
0,2573208,48+43+34=125,125,12,"[6, 10, 15, 6, 5, 15, 5, 6, 17, 3, 4, 7, 1]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 4, ..."
1,1630340,30-(48+13)=-31,-31,14,"[5, 2, 16, 12, 6, 10, 15, 3, 5, 13, 17, 16, 5,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 16, 5,..."
2,549277,(21*31)+10=661,661,14,"[12, 4, 3, 14, 5, 3, 13, 15, 3, 2, 17, 8, 8, 3...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 8, 8, ..."
3,133957,2-27-10=-35,-35,11,"[4, 16, 4, 9, 16, 3, 2, 17, 16, 5, 7, 1]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 16, 5,..."
4,1279828,(15*20)+24=324,324,14,"[12, 3, 7, 14, 4, 2, 13, 15, 4, 6, 17, 5, 4, 6...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 5, 4, ..."


# Hyper Parameters

|Hyperparameter|Meaning|Value|
|-|-|-|
|`batch_size`|Number of data samples in a single batch|64|
|`epochs`|Total number of epochs to train|10|
|`embed_dim`|Dimension of the word embeddings|256|
|`hidden_dim`|Dimension of the hidden state in each timestep of the LSTM|256|
|`lr`|Learning Rate|0.001|
|`grad_clip`|To prevent gradient explosion in RNNs, restrict the gradient range|1|

In [59]:
batch_size = 64
epochs = 2
embed_dim = 256
hidden_dim = 256
lr = 0.001
grad_clip = 1

# Data Batching
- Use `torch.utils.data.Dataset` to create a data generation tool called  `dataset`.
- The, use `torch.utils.data.DataLoader` to randomly sample from the `dataset` and group the samples into batches.

In [11]:
class Dataset(torch.utils.data.Dataset):
    def __init__(self, sequences):
        self.sequences = sequences
    
    def __len__(self):
        # return the amount of data
        return len(self.sequences)
    
    def __getitem__(self, index):
        # Extract the input data x and the ground truth y from the data
        x = self.sequences[index][0]
        y = self.sequences[index][1]
        return x, y

# collate function, used to build dataloader
def collate_fn(batch):
    batch_x = [torch.tensor(data[0]) for data in batch]
    batch_y = [torch.tensor(data[1]) for data in batch]
    batch_x_lens = torch.LongTensor([len(x) for x in batch_x])
    batch_y_lens = torch.LongTensor([len(y) for y in batch_y])
    
    # Pad the input sequence
    pad_batch_x = torch.nn.utils.rnn.pad_sequence(batch_x,
                                                  batch_first=True,
                                                  padding_value=char_to_id['<pad>'])
    
    pad_batch_y = torch.nn.utils.rnn.pad_sequence(batch_y,
                                                  batch_first=True,
                                                  padding_value=char_to_id['<pad>'])
    
    return pad_batch_x, pad_batch_y, batch_x_lens, batch_y_lens

In [12]:
ds_train = Dataset(df_train[['char_id_list', 'label_id_list']].values.tolist())
ds_eval = Dataset(df_eval[['char_id_list', 'label_id_list']].values.tolist())

In [13]:
import torch
from torch.utils.data import DataLoader

In [27]:
# Build dataloader of train set and eval set, collate_fn is the collate function
dl_train = DataLoader(ds_train, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)
dl_eval = DataLoader(ds_eval, batch_size=batch_size, shuffle=False, collate_fn=collate_fn)

Batch size: 64
Batch size: 64


# Model Design

## Execution Flow
1. Convert all characters in the sentence into embeddings.
2. Pass the embeddings through an LSTM sequentially.
3. The output of the LSTM is passed into another LSTM, and additional layers can be added.
4. The output from all time steps of the final LSTM is passed through a Fully Connected layer.
5. The character corresponding to the maximum value across all output dimensions is selected as the next character.

## Loss Function
Since this is a classification task, Cross Entropy is used as the loss function.

## Gradient Update
Adam algorithm is used for gradient updates.

In [15]:
class CharRNN(torch.nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_dim):
        super(CharRNN, self).__init__()
        
        self.embedding = torch.nn.Embedding(num_embeddings=vocab_size,
                                            embedding_dim=embed_dim,
                                            padding_idx=char_to_id['<pad>'])
        
        self.rnn_layer1 = torch.nn.LSTM(input_size=embed_dim,
                                        hidden_size=hidden_dim,
                                        batch_first=True)
        
        self.rnn_layer2 = torch.nn.LSTM(input_size=hidden_dim,
                                        hidden_size=hidden_dim,
                                        batch_first=True)
        
        self.linear = torch.nn.Sequential(torch.nn.Linear(in_features=hidden_dim,
                                                          out_features=hidden_dim),
                                          torch.nn.ReLU(),
                                          torch.nn.Linear(in_features=hidden_dim,
                                                          out_features=vocab_size))
        
    def forward(self, batch_x, batch_x_lens):
        return self.encoder(batch_x, batch_x_lens)
    
    # The forward pass of the model
    def encoder(self, batch_x, batch_x_lens):
        batch_x = self.embedding(batch_x)
        
        batch_x = torch.nn.utils.rnn.pack_padded_sequence(batch_x,
                                                          batch_x_lens,
                                                          batch_first=True,
                                                          enforce_sorted=False)
        
        batch_x, _ = self.rnn_layer1(batch_x)
        batch_x, _ = self.rnn_layer2(batch_x)
        
        batch_x, _ = torch.nn.utils.rnn.pad_packed_sequence(batch_x,
                                                            batch_first=True)
        
        batch_x = self.linear(batch_x)
        
        return batch_x
    
    def generator(self, start_char, max_len=200):
        
        char_list = [char_to_id[c] for c in start_char]
        
        next_char = None
        
        while len(char_list) < max_len: 
            # Write your code here 
            # Pack the char_list to tensor
            # Input the tensor to the embedding layer, LSTM layers, linear respectively
            
            # y = # Obtain the next token prediction y
            # 將 char_list 轉換為 tensor
            input_tensor = torch.tensor(char_list).unsqueeze(0)  # 增加 batch 維度
            
            # 獲取序列長度
            input_len = torch.tensor([len(char_list)])
            
            # 輸入到嵌入層和 LSTM 層
            y = self.forward(input_tensor, input_len)
            
            # 獲取最後一個時間步的輸出
            y = y[0, -1, :]
            next_char = torch.argmax(y).item() # Use argmax function to get the next token prediction
            
            if next_char == char_to_id['<eos>']:
                break
            
            char_list.append(next_char)
            
        return [id_to_char[ch_id] for ch_id in char_list]

In [87]:
import torch

torch.manual_seed(2)

device = torch.device("cpu") # Specify a device (cpu)

model = CharRNN(vocab_size,
                embed_dim,
                hidden_dim).to(device)

In [17]:
criterion = torch.nn.CrossEntropyLoss(ignore_index=char_to_id['<pad>'])# Write your code here. Cross-entropy loss function. The loss function should ignore <pad>
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)# Write your code here. Use Adam or AdamW for Optimizer

# Training
1. The outer `for` loop controls the `epoch`
    1. The inner `for` loop uses `data_loader` to retrieve batches.
        1. Pass the batch to the `model` for training.
        2. Compare the predicted results `batch_pred_y` with the true labels `batch_y` using Cross Entropy to calculate the loss `loss`
        3. Use `loss.backward` to automatically compute the gradients.
        4. Use `torch.nn.utils.clip_grad_value_` to limit the gradient values between `-grad_clip` &lt; and &lt; `grad_clip`.
        5. Use `optimizer.step()` to update the model (backpropagation).
2.  After every `1000` batches, output the current loss to monitor whether it is converging.

In [60]:
from tqdm import tqdm
from copy import deepcopy

best_model = None
best_em = 0

model = model.to(device)
model.train()
i = 0

for epoch in range(1, epochs+1):
    # The process bar
    bar = tqdm(dl_train, desc=f"Train epoch {epoch}")
    for batch_x, batch_y, batch_x_lens, batch_y_lens in bar:
        # Clear the gradient
        optimizer.zero_grad()

        batch_pred_y = model(batch_x.to(device), batch_x_lens)
        
        # Input the prediction and ground truths to loss function
        # Back propagation
        batch_pred_y = batch_pred_y.view(-1, vocab_size)
        batch_y = batch_y.reshape(-1).to(device).view(-1)

        # Check if batch_pred_y and batch_y have the same shape
        if batch_pred_y.shape[0] != batch_y.shape[0]:
            min_len = min(batch_pred_y.shape[0], batch_y.shape[0])
            batch_pred_y = batch_pred_y[:min_len]
            batch_y = batch_y[:min_len]

        loss = criterion(batch_pred_y, batch_y)

        # 反向傳播
        loss.backward()
        
        torch.nn.utils.clip_grad_value_(model.parameters(), grad_clip) # gradient clipping

        # Optimize parameters in the model
        optimizer.step()

        i += 1
        if i % 50 == 0:
            bar.set_postfix(loss = loss.item())
    
    # 驗證過程
    model.eval()

    # Evaluate your model
    bar = tqdm(dl_eval, desc=f"Validation epoch {epoch}")
    matched = 0
    total = 0
    with torch.no_grad():  # 禁用梯度計算
        for batch_x, batch_y, batch_x_lens, batch_y_lens in bar:
        
            predictions = model(batch_x.to(device), batch_x_lens)
            predictions = predictions.argmax(dim=-1)
            
            # 檢查預測是否與真實標籤匹配
            batch_y = batch_y.to(device)

            # 確保 predictions 和 batch_y 的形狀匹配
            if predictions.shape[0] != batch_y.shape[0]:
                min_len = min(predictions.shape[0], batch_y.shape[0])
                predictions = predictions[:min_len]
                batch_y = batch_y[:min_len]

                matched += (predictions == batch_y).sum().item()
            total += batch_y.numel()
            
    em = matched / total
    print(f"Exact Match (EM) at epoch {epoch}: {em:.4f}")
    
    # 保存最佳模型
    if em > best_em:
        best_em = em
        best_model = deepcopy(model)

    # 恢復訓練模式
    model.train()

Train epoch 1:   0%|          | 94/37020 [00:04<26:55, 22.85it/s, loss=2.9]

# Generation
Use `model.generator` and provide an initial character to automatically generate a sequence.

In [86]:
model = model.to('cpu')
print("".join(model.generator('1+1=')))

RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu! (when checking argument for argument index in method wrapper_CUDA__index_select)